# Contents : 

This was done in a team with mohitgupta_omg.


1. Language Influences Page Total View 

2. Checking the stationarity
    2.1 Plotting rolling statistics 
    2.2 Dickey-Fuller test
    
3. Making the time series stationery
        - Estimitating and eliminating trend and seasonality
3.1. Log transformation
3.2. Remove trend and seasonality with decomposition
3.3. Remove trend and seasonality with differencing
    
4. Plot the autocorreltaion and partial auto correlation functions

**Applying Models:** 
    
5. Auto Regressive (AR) Model  
6. Moving Average (MA) Model
7. ARIMA MODEL

## Augmented Dicky Fuller Test:

**Augmented Dicky Fuller Test:**
The Augmented Dicky Fuller test is a type of statistical test called a unit root test.
The intuition behind a unit root test is that it determines how strongly a time series is
defined by a trend.
There are no. of unit root tests and ADF may be one of the most widely used
1. Null Hypothesis (H0): Null hypothesis of the test is that the time series can
be represented by a unit root that is not stationary.
2. Alternative Hypothesis (H1): Alternative Hypothesis of the test is that the
time series is stationary.

**Interpretation of ADF Value:**
1. p value &gt; 0.05: Accepts the Null Hypothesis (H0), the data has a unit root and
is non-stationary.
2. p value &lt; = 0.05: Rejects the Null Hypothesis (H0), the data is stationary.

In [ ]:
import pandas as kunfu
import numpy as dragon
import pylab as p
import matplotlib.pyplot as plot
from collections import Counter
import re

In [ ]:
train = kunfu.read_csv('../input/train_1.csv').fillna(0)
print(train.head())

x = train[['Page']].values
print("\nTotal No. of Articles : ", (len(x)), "\n")

In [ ]:
print(train.info())

# Language Influences Page Total View  <(0_o)>

In [ ]:
# Using Counters 
# dictionary keys and their counts are stored as dictionary values
# in a sorted manner as per value

# Reason why we are taking 'en' as a root language to focus on
def lang(Page):
    val = re.search('[a-z][a-z].wikipedia.org',Page)
    if val:
        return val[0][0:2]           
    
    # no_lang for media files ; wikimedia.org
    return 'no_lang'

train['language'] = train.Page.map(lang)

# Article Count 
print("\nArticle count as per Language : \n", Counter(train.language))

In [ ]:

language_set = {}
language_set['en'] = train[train.language=='en'].iloc[:,0:-1]
language_set['ja'] = train[train.language == 'ja'].iloc[:, 0:-1]
language_set['de'] = train[train.language == 'de'].iloc[:, 0:-1]
language_set['fr'] = train[train.language == 'fr'].iloc[:, 0:-1]
language_set['ru'] = train[train.language == 'ru'].iloc[:, 0:-1]
language_set['es'] = train[train.language == 'es'].iloc[:, 0:-1]
language_set['no_lang'] = train[train.language == 'no_lang'].iloc[:, 0:-1]

for key in language_set:
    print("KEY : ", language_set[key],"\n")

In [ ]:
# axis =0 : vertical in NumPy ;   axis =1 : horizontal in NumPy
total_view = {} 
for key in language_set:
    total_view[key] = language_set[key].iloc[:, 1:].sum(axis=0) / language_set[key].shape[0]


for key in language_set:
    print("KEY : ", key)
    print("\nTotal_Value KEY : \n", total_view[key])

In [ ]:
# Still not clear
days = [r
        for r in range(total_view['en'].shape[0])]

# height and width of graph
plot.figure(figsize=(8, 6))
labels={'ja':'Japanese','de':'German','en' : 'English','no_lang':'Media_File','fr':'French','zh':'Chinese',
        'ru':'Russian','es':'Spanish'}

for key in total_view:
    plot.plot(days,total_view[key],label = labels[key])
    

plot.ylabel('Views per Page')
plot.xlabel('Days (2015-07-1  to  2016-12-31)')
plot.title('Language Influences Page Total_View\nCheck it out  <(0_)>')
plot.legend(loc = 'upper left', bbox_to_anchor = (1.2, 1))
plot.show()

In [ ]:
plot.plot(days,total_view['en'],label=labels['en'])
plot.show()

# Checking the stationarity 

1.Plotting rolling statistics
2.Dickey-Fuller test

Here the null hypothesis is that the TS is non-stationary.
The test results comprise of a Test Statistic and some Critical Values for difference confidence levels.
If the ‘Test Statistic’ is less than the ‘Critical Value’, we can reject the null hypothesis and say 
that the series is stationary. 



In [ ]:


from statsmodels.tsa.stattools import adfuller

def test_stationarity(x):


    #Determing rolling statistics
    rolmean = x.rolling(window=22,center=False).mean()

    rolstd = x.rolling(window=12,center=False).std()
    
    #Plot rolling statistics:
    orig = plot.plot(x.values, color='blue',label='Original')
    mean = plot.plot(rolmean.values, color='red', label='Rolling Mean')
    std = plot.plot(rolstd.values, color='black', label = 'Rolling Std')
    plot.legend(loc='best')
    plot.title('Rolling Mean & Standard Deviation')
    plot.show(block=False)
    
    #Perform Dickey Fuller test    
    result=adfuller(x)
    print('ADF Stastistic: %f'%result[0])
    print('p-value: %f'%result[1])
    pvalue=result[1]
    for key,value in result[4].items():
         if result[0]>value:
            print("The graph is non stationery")
            break
         else:
            print("The graph is stationery")
            break;
    print('Critical values:')
    for key,value in result[4].items():
        print('\t%s: %.3f' % (key, value))
        
test_stationarity(total_view['en'])


# Making the time series stationery

## Estimitating and eliminating trend and seasonality


## 1.Log transformation

In [ ]:

ts_log = dragon.log(total_view['en'])
plot.plot(ts_log.values,color="green")
plot.show()

test_stationarity(ts_log)

# Remove trend and seasonality with decomposition

In [ ]:
# Naive decomposition of our Time Series as explained above
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(ts_log.values, model='multiplicative',freq = 7)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid


plot.subplot(411)
plot.title('Obeserved = Trend + Seasonality + Residuals')
plot.plot(ts_log.values,label='Observed')
plot.legend(loc='best')
plot.subplot(412)
plot.plot(trend, label='Trend')
plot.legend(loc='best')
plot.subplot(413)
plot.plot(seasonal,label='Seasonality')
plot.legend(loc='best')
plot.subplot(414)
plot.plot(residual, label='Residuals')
plot.legend(loc='best')
plot.tight_layout()
plot.show()

In [ ]:
ts_log_decompose = residual
#ts_log_decompose.fillna(inplace=True)
#test_stationarity(ts_log_decompose)

# Remove trend and seasonality with differencing


In [ ]:
ts_log_diff = ts_log - ts_log.shift()
plot.plot(ts_log_diff.values)
plot.show()

In [ ]:
ts_log_diff.dropna(inplace=True)
test_stationarity(ts_log_diff)


The ADF statistic is much lesser than critical value at 1%. So there is 99% confidence interval that our graph is now stationery.
Now we can apply the ARIMA model

# Plot the autocorreltaion and partial auto correlation functions

In [ ]:
#ACF and PACF plots
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA

lag_acf = acf(ts_log_diff, nlags=10)
lag_pacf = pacf(ts_log_diff, nlags=10, method='ols')

#Plot ACF: 
plot.subplot(1,1,1)

plot.plot(lag_acf)
#print(lag_acf[0.5])

plot.axhline(y=0,linestyle='--',color='g')
plot.title('Autocorrelation Function')
plot.show()

#Plot PACF:
plot.subplot(1,1,1)
plot.plot(lag_pacf)

plot.axhline(y=0,linestyle='--',color='green')
plot.title('Partial Autocorrelation Function ')
plot.tight_layout()
plot.show()


# Auto Regressive (AR) Model

In [ ]:
# follow lag
model = ARIMA(ts_log.values, order=(1,1,0))  
results_ARIMA = model.fit(disp=-1)  
plot.plot(ts_log_diff.values)
plot.plot(results_ARIMA.fittedvalues, color='red')
plot.title('RSS: %.7f'% sum((results_ARIMA.fittedvalues-ts_log_diff.values)**2))
plot.show()

# Moving Average (MA) Model

In [ ]:
# follow error
model = ARIMA(ts_log.values, order=(0,1,1))  
results_ARIMA = model.fit(disp=-1)  
plot.plot(ts_log_diff.values)
plot.plot(results_ARIMA.fittedvalues, color='red')
plot.title('RSS: %.7f'% sum((results_ARIMA.fittedvalues-ts_log_diff.values)**2))
plot.show()

# ARIMA MODEL

In [ ]:
model = ARIMA(ts_log.values, order=(1,1,1))  
results_ARIMA = model.fit(disp=-1)  
plot.plot(ts_log_diff.values)
plot.plot(results_ARIMA.fittedvalues, color='red')
plot.title('RSS: %.7f'% sum((results_ARIMA.fittedvalues-ts_log_diff.values)**2))
plot.show()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from pandas import DataFrame
print(results_ARIMA.summary())

In [ ]:
size = int(len(ts_log)-100)
train_arima, test_arima = ts_log[0:size], ts_log[size:len(ts_log)]
history = [x for x in train_arima]
predictions = list()
originals = list()
error_list = list()

print('Printing Predicted vs Expected Values...')
print('\n')
for t in range(len(test_arima)):
    model = ARIMA(history, order=(1, 1, 1))
    model_fit = model.fit(disp=0)
    
    output = model_fit.forecast()
    
    pred_value = output[0]
    
        
    original_value = test_arima[t]
    history.append(original_value)
    
    pred_value = dragon.exp(pred_value)
    
    
    original_value = dragon.exp(original_value)
    
    
    error = ((abs(pred_value - original_value)) / original_value) * 100
    error_list.append(error)
    print('predicted = %f,   expected = %f,   error = %f ' % (pred_value, original_value, error), '%')
    
    predictions.append(float(pred_value))
    originals.append(float(original_value))
    
    #error = mean_squared_error(dragon.exp(yhat), dragon.exp(obs))
    #print('mean_squared_error : ', error)
print('\n Means Error in Predicting Test Case Articles : %f ' % (sum(error_list)/float(len(error_list))), '%')


In [ ]:
plot.figure(figsize=(8, 6))
test_day = [t+450
           for t in range(len(test_arima))]
labels={'Orginal','Predicted'}
plot.plot(test_day, predictions, color= 'green')
plot.plot(test_day, originals, color = 'orange')
plot.title('Expected Vs Predicted Views Forecasting _<(0_o)>')
plot.xlabel('Days')
plot.ylabel('Total Views')
plot.legend(labels)
plot.show()

In [ ]:
plot.figure(figsize=(8, 6))
test_day = [t+450
           for t in range(len(test_arima))]
labels={'Orginal','Predicted'}
plot.plot(test_day, predictions, color= 'red')
plot.plot(days, total_view['en'], color = 'orange')
plot.title('Expected Vs Predicted Views Forecasting _<(0_o)>')
plot.xlabel('Days')
plot.ylabel('Total Views')
plot.legend(labels)
plot.show()

In [ ]:
npages = 5
top_pages = {}
key = 'en'
print(key)
sum_set = kunfu.DataFrame(language_set[key][['Page']])
sum_set['total'] = language_set[key].sum(axis=1)
sum_set = sum_set.sort_values('total',ascending=False)
print(sum_set.head(5))
top_pages[key] = sum_set.index[0]
print('\n')

In [ ]:
def plot_entry(key,idx):
    data = language_set[key].iloc[idx,1:]
    fig = plot.figure(1,figsize=(10,5))
    plot.plot(days,data)
    plot.xlabel('day')
    plot.ylabel('views')
    plot.title(train.iloc[language_set[key].index[idx],0])
    
    plot.show()
    
idx = [1, 2, 3, 4, 5]
for i in idx:
    plot_entry('en',i)

In [ ]:

data = language_set['en'].iloc[1,1:]
fig = plot.figure(1,figsize=(10,5))
plot.plot(days,data)
plot.xlabel('day')
plot.ylabel('views')
plot.title(train.iloc[language_set['en'].index[1],0])
plot.show()

print(train.iloc[language_set['en'].index[1],0])
print(data.tail())

#               Means Error in Predicting Test Case Articles : 5.420564  %

Thanks for having a look. Please like the KERNEL